### Load Model from Google Drive


In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
import keras

from keras.models import Model, load_model


from keras.layers import Lambda
import os
# authorization
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [0]:
#softmax layer
def soft_max(x):
    import tensorflow as tf
    return tf.nn.softmax(x)

In [3]:
file_id = '1cDvam2G2hilZW62H_-ApMFXXMH7PDdn6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model.h5')
lmodel = load_model('model.h5')

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
from keras import optimizers
rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
x = lmodel.output
predictions = Lambda(lambda x: soft_max(x))(x)
lmodel = Model(inputs=lmodel.input, outputs=predictions)
lmodel.compile(optimizer=rms, loss='categorical_crossentropy')

In [0]:
#load picture
file_id = '1cH1JpRvW6YgT3cXHLOGWG5BwSs-clshA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test.png')
#import cv2
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
test_img = cv2.imread('test.png')
test_gcimg = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

In [41]:

test_img = cv2.resize(test_gcimg,dsize=(28,28),interpolation=cv2.INTER_LINEAR)
test_img = test_img.astype('float32')
test_img *= (1/255)
test_img = test_img.reshape((1,)+test_img.shape + (1,))
test_img.shape

(1, 28, 28, 1)

In [42]:
lmodel.predict(test_img)

array([[0., 0., 0., 1.]], dtype=float32)